
  # Introduction to Deep Learning (LDA-T3114)
  ## Code for Assignment 1: Sentiment Classification on a Feed-Forward Neural Network

   Group ID: **GradientDescendants**
   Group Members:
   * Cardin Christian
   * Nieminen Harri
   * Zetterman Elina


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from data_semeval import *
from paths import data_dir

In [19]:
#--- hyperparameters ---

N_CLASSES = len(LABEL_INDICES)
N_EPOCHS = 20
LEARNING_RATE = 0.02
BATCH_SIZE = 20
REPORT_EVERY = 1
IS_VERBOSE = False

In [13]:
def make_bow(tweet, indices):
    feature_ids = list(indices[tok] for tok in tweet['BODY'] if tok in indices)
    bow_vec = torch.zeros(len(indices))
    bow_vec[feature_ids] = 1
    return bow_vec.view(1, -1)

def generate_bow_representations(data):
    vocab = set(token for tweet in data['training'] for token in tweet['BODY'])
    vocab_size = len(vocab) 
    indices = {w:i for i, w in enumerate(vocab)}
  
    for split in ["training","development.input","development.gold",
                  "test.input","test.gold"]:
        for tweet in data[split]:
            tweet['BOW'] = make_bow(tweet,indices)

    return indices, vocab_size

# Convert string label to pytorch format.
def label_to_idx(label):
    return torch.LongTensor([LABEL_INDICES[label]])

In [14]:
#--- data loading ---
data = read_semeval_datasets(data_dir)
indices, vocab_size = generate_bow_representations(data)

In [15]:
class FFNN(nn.Module):
    def __init__(self, input_size: int, n_classes: int, hidden_layers: int, layer_size: int):
        """
        hidden_layers: number of hidden layers in the network
        layer_size: number of neurons in each hidden layer
        """
        super(FFNN, self).__init__()
        self.in_layer = nn.Linear(in_features=input_size, out_features=layer_size)

        # Note: the range is up to (hidden_layers - 1) because the first layer is also a hidden layer
        self.hidden_layers = [nn.Linear(in_features=layer_size, out_features=layer_size) for _ in range(hidden_layers - 1)]
        self.out_layer = nn.Linear(in_features=layer_size, out_features=n_classes)

    def forward(self, x):
        # Uses ReLu as nonlinear function for hidden layers
        result = torch.relu(self.in_layer(x))
        for layer in self.hidden_layers:
            result = torch.relu(layer(result))

        # Important: need to specify the dimension over which to compute the softmax.
        return F.log_softmax(self.out_layer(result), dim=1)

In [22]:
model = FFNN(vocab_size, N_CLASSES, hidden_layers=2, layer_size=15)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [23]:
#--- training ---
for epoch in range(N_EPOCHS):
    total_loss = 0
    #shuffle the datasets per every epoch
    random.shuffle(data['training'])

    for i in range(int(len(data['training'])/BATCH_SIZE)):
        minibatch = data['training'][i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        
        # run all inputs in the batch through the model to get the predictions
        # optimize loss function with batch predictions and actuals
        pred = model(torch.vstack([tweet["BOW"] for tweet in minibatch]))
        act = torch.tensor([label_to_idx(tweet["SENTIMENT"]) for tweet in minibatch])  
            
        # loss calculation and optimisation for batch
        loss = loss_function(pred, act)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update total loss for epoch
        total_loss += loss.item()

    if ((epoch+1) % REPORT_EVERY) == 0:
        print('epoch: %d, loss: %.4f' % (epoch+1, total_loss*BATCH_SIZE/len(data['training'])))


epoch: 1, loss: 1.0073
epoch: 2, loss: 0.9878
epoch: 3, loss: 0.9799
epoch: 4, loss: 0.9625
epoch: 5, loss: 0.9341
epoch: 6, loss: 0.9008
epoch: 7, loss: 0.8686
epoch: 8, loss: 0.8351
epoch: 9, loss: 0.8019
epoch: 10, loss: 0.7670
epoch: 11, loss: 0.7324
epoch: 12, loss: 0.6996
epoch: 13, loss: 0.6653
epoch: 14, loss: 0.6342
epoch: 15, loss: 0.6033
epoch: 16, loss: 0.5719
epoch: 17, loss: 0.5424
epoch: 18, loss: 0.5134
epoch: 19, loss: 0.4851
epoch: 20, loss: 0.4572


In [24]:
#--- test ---
correct = 0
test_len = len(data['test.gold'])
with torch.no_grad():
    for tweet in data['test.gold']:
        gold_class = label_to_idx(tweet['SENTIMENT'])
        predicted = model(tweet["BOW"]).argmax()

        if predicted == gold_class:
            correct += 1

        if IS_VERBOSE:
            print('TEST DATA: %s, GOLD LABEL: %s, GOLD CLASS %d, OUTPUT: %d' %
                 (' '.join(tweet['BODY'][:-1]), tweet['SENTIMENT'], gold_class, predicted))

    print(f"Test Accuracy: {correct}/{test_len} {correct/test_len:.2%}")

Test Accuracy: 3055/4633 65.94%
